In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
import sys
sys.path.insert(0, os.path.abspath("../.."))
from utils import time, data_processor, constants

In [2]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu_27_countries = constants.eu
balkans = constants.balkans
balkans

['BA', 'KV', 'AL', 'RS', 'ME']

In [3]:
KEY_FINANCIALS_DETAILED_INDUSTRY_PATH = "..\\..\\data_raw\\key_financials_detailed\\industry_global_financials_and_ratios_eur"
KEY_FINANCIALS_DETAILED_BANKSS_PATH = "..\\data_raw\\key_financials_detailed\\banks_global_financials_and_ratios_eur"
KEY_FINANCIALS_DETAILED_INSURANCES_PATH = "..\\data_raw\\key_financials_detailed\\insurances_global_financials_and_ratios_eur"

In [4]:
key_financials_detail_path = "..\\..\\data_raw\\key_financials_detailed\\industry_global_financials_and_ratios_eur\\"

def fetch_data_for_country(country_iso_code):
    query = f"""
    SELECT 
        bvd_id_number,
        operating_revenue_turnover_,
        number_of_employees,
        costs_of_goods_sold,
        material_costs,
        added_value,
        EXTRACT(YEAR FROM closing_date) AS year  
    FROM 
        parquet_scan('{key_financials_detail_path}*.parquet')
    WHERE
        CAST(bvd_id_number AS STRING) LIKE '{country_iso_code}%'
    ORDER BY 
        bvd_id_number, year
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()

    return df


In [5]:
# TEST
# it = fetch_data_for_country("IT")
# ba = fetch_data_for_country("BA")
# cs = fetch_data_for_country("CS")
# mk = fetch_data_for_country("MK")
# xk = fetch_data_for_country("XK")
# it = fetch_data_for_country("IT")


In [8]:
import os

output_path = "..\\..\\data_processed\\key_financials_detailed_processed\\"

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

for country in all_countries:
    df = fetch_data_for_country(country)
    df.to_parquet(f"{output_path}key_financials_detailed_{country}.parquet")
    print(f"Saved: {output_path}key_financials_detailed_{country}.parquet")


Saved: ..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_BA.parquet
Saved: ..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_KV.parquet
Saved: ..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_AL.parquet
Saved: ..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_RS.parquet
Saved: ..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_ME.parquet


In [9]:
df

,bvd_id_number,operating_revenue_turnover_,number_of_employees,costs_of_goods_sold,material_costs,added_value,year
0,ME00200838,9979000.0,738.0,2078000.0,2051000.0,6508000.0,2012
1,ME00200838,10983000.0,721.0,2159000.0,2119000.0,7019000.0,2013
2,ME00200838,10673000.0,NaN,2115000.0,2080000.0,6909000.0,2014
3,ME00200838,9944000.0,651.0,1975000.0,1944000.0,6528000.0,2015
4,ME00200838,9882000.0,682.0,1970000.0,1940000.0,6378000.0,2016
...,...,...,...,...,...,...,...
147716,ME40098E2,NaN,1.0,0.0,NaN,-2000.0,2019
147717,ME40098E2,NaN,1.0,NaN,NaN,-1000.0,2020
147718,ME40098E2,NaN,NaN,NaN,NaN,0.0,2021
147719,ME40098E2,NaN,NaN,NaN,NaN,-2000.0,2022


In [10]:
## TEST 

# firmographics_ita = pd.read_parquet("..\\..\\data_processed\\firmographics_processed\\firmographics_IT.parquet")

In [10]:
# merged_df = pd.merge(
#     firmographics_ita,
#     it,
#     left_on="bvd_id_number",
#     right_on="bvd_id_number",
#     how="outer",         
#     indicator=True       
# )


In [11]:
# merged_df.value_counts("_merge")

_merge
both          75223744
left_only      1280343
right_only      151814
Name: count, dtype: int64